# Red Wine Quality

Kaggle link: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import wandb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv


First, we need to import Pytorch

In [2]:
import torch
from torch import nn
from torch import optim
from torch.utils import data
import wandb
wandb.init(project="simple_regression")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Data Processing

In [3]:
train_data = pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
train_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


We need to separate features from target

In [4]:
n_train = train_data.shape[0]
all_features = train_data.iloc[:, 1:-1]
all_features = all_features.apply(lambda x: (x - x.mean()) / (x.std()))
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32)
train_features

tensor([[ 0.9616, -1.3910, -0.4531,  ...,  1.2882, -0.5790, -0.9599],
        [ 1.9668, -1.3910,  0.0434,  ..., -0.7197,  0.1289, -0.5846],
        [ 1.2967, -1.1857, -0.1694,  ..., -0.3311, -0.0481, -0.5846],
        ...,
        [-0.0995, -0.7237, -0.1694,  ...,  0.7053,  0.5419,  0.5415],
        [ 0.6544, -0.7750, -0.3822,  ...,  1.6769,  0.3059, -0.2092],
        [-1.2165,  1.0217,  0.7527,  ...,  0.5110,  0.0109,  0.5415]])

In [5]:
trains_labels = train_data.quality.values.reshape(-1, 1)
trains_labels = torch.tensor(trains_labels,dtype = torch.float)
trains_labels

tensor([[5.],
        [5.],
        [5.],
        ...,
        [6.],
        [5.],
        [6.]])

In [6]:
trains_mean = trains_labels.mean()
trains_std = trains_labels.std()
trains_labels = (trains_labels - trains_mean) / trains_std
train_labels = torch.tensor(trains_labels,
                            dtype=torch.float32)
train_labels

/tmp/ipykernel_32/2511893988.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(trains_labels,


tensor([[-0.7876],
        [-0.7876],
        [-0.7876],
        ...,
        [ 0.4507],
        [-0.7876],
        [ 0.4507]])

## Training

Initialize the weight of the linear regression

In [7]:
n_features = train_features.shape[1]
linear_weights = torch.randn(n_features, requires_grad=True)


Training loop

In [11]:
num_iterations = 200
lr = 0.05 # I change that because I found that this lr I can get less err after 512 iier
linear_weights = torch.randn(n_features, requires_grad=True) # reset for each running 
criterion = nn.MSELoss()
for i in range(num_iterations):
    prediction = torch.matmul(train_features, linear_weights)
    prediction = prediction.reshape(-1,1)
    loss = criterion(prediction,trains_labels)
    loss.backward()
    with torch.no_grad():
        linear_weights = linear_weights - lr * linear_weights.grad
        wandb.log({'loss': loss.item()})
    linear_weights.requires_grad = True
    
# The reason why I put it here, because Kaggle sometimes doesnt indicate me it is finished or still running    
print("finish") 
loss.item()

finish


0.639532744884491

Get the real predictions

In [12]:
predictions = (prediction * trains_std) + trains_mean
predictions = torch.round(predictions) # I put round function because by dataset output is number from 0 to 10 of integer type
predictions

tensor([[5.],
        [5.],
        [5.],
        ...,
        [6.],
        [5.],
        [6.]], grad_fn=<RoundBackward0>)

Our predictions seem very close to the ground truth!

**To go further**: Stochastic Gradient Descent is not the optimal algorithm in terms of convergeance.
If you are curious, you can read this nice article about an improvement to SGD, momentum and try to implement it: https://distill.pub/2017/momentum/